<a href="https://colab.research.google.com/github/JuliuszCzupryniak/Distorted-patterns-recognition-in-text/blob/main/Regex_recognition_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [3]:
def get_unique_chars(data):
  temp_list=""
  for i in data.text:
    temp_list=temp_list+i
  chars = list(set(temp_list))
  chars.sort()
  return chars

def category_encoding(datatarget):
  outputs = []
  outputs.append("")
  for i in range(len(datatarget)):
    for j in range(len(datatarget[i])):
      outputs.append(datatarget[i][j])
  outputs=list(set(outputs))
  outputs.sort()
  temp_list=[]
  x=0;
  for i in outputs:
    temp_list.append(x)
    x+=1
  outputs = pd.DataFrame({'Output':outputs,'Label':temp_list})
  return outputs

def standarize_data_length(data,size):
  for i in data:
    for j in range(size-len(i)):
      i.append("")
  return data

def text_to_list(data):
  dataset=[]
  temp_list=[]
  for i in data.text:
    for j in i:
      temp_list.append(j)
    dataset.append(temp_list.copy())
    temp_list.clear()
  return dataset

def regex_to_list(data):
  datatarget=[]
  temp_list=[]
  for i in range(len(data.regex)):
    for j in range(len(data.regex[i])):
      output=""
      if data.regex[i][j]=="[":
        j+=1
        while data.regex[i][j] != "]":
          output=output+data.regex[i][j]
          j+=1
        temp_list.append(output)
    datatarget.append(temp_list.copy())
    temp_list.clear()
  return datatarget

def standarize_target_vector(datatarget,outputs):
  for i in range(len(datatarget)):
    for j in range(len(datatarget[i])):
      for k in range(len(outputs)):
        if datatarget[i][j]==outputs.Output[k]:
          datatarget[i][j]=outputs.Label[k]
          break
  return datatarget

In [4]:
train_data = pd.read_csv("/content/train_data.csv")
test_data = pd.read_csv("/content/test_data.csv")
predict_data = pd.read_csv("/content/predict_data.csv")
display_side_by_side(train_data[0:10],test_data[:10],predict_data[:10])
print()
print("Maksymalna długość obserwacji w zbiorze treningowym  :  " + str(max(len(x) for x in train_data.text)))
print("Maksymalna długość obserwacji w zbiorze testowym     :  " + str(max(len(x) for x in test_data.text)))
print("Maksymalna długość obserwacji w zbiorze do predykcji :  " + str(max(len(x) for x in predict_data.text)))

,text,regex
0,47-199,[0-9][0-9][-][0-9][0-9][0-9]
1,01-332,[0-9][0-9][-][0-9][0-9][0-9]
2,48-858,[0-9][0-9][-][0-9][0-9][0-9]
3,36-059,[0-9][0-9][-][0-9][0-9][0-9]
4,75-388,[0-9][0-9][-][0-9][0-9][0-9]
5,16-196,[0-9][0-9][-][0-9][0-9][0-9]
6,52-111,[0-9][0-9][-][0-9][0-9][0-9]
7,61-240,[0-9][0-9][-][0-9][0-9][0-9]
8,90-111,[0-9][0-9][-][0-9][0-9][0-9]
9,86-563,[0-9][0-9][-][0-9][0-9][0-9]



Maksymalna długość obserwacji w zbiorze treningowym  :  9
Maksymalna długość obserwacji w zbiorze testowym     :  8
Maksymalna długość obserwacji w zbiorze do predykcji :  9


In [5]:
alphabet = get_unique_chars(train_data)
alphabet_lookup = category_encoding(alphabet)

display_side_by_side(alphabet_lookup[0:16],alphabet_lookup[16:32],alphabet_lookup[32:48],alphabet_lookup[48:64],alphabet_lookup[64:80],alphabet_lookup[80:96],alphabet_lookup[96:112],alphabet_lookup[112:])

,Output,Label
0,,0
1,-,1
2,0,2
3,1,3
4,2,4
5,3,5
6,4,6
7,5,7
8,6,8
9,7,9


In [6]:
train_set = text_to_list(train_data)
test_set= text_to_list(test_data)
predict_set = text_to_list(predict_data)

In [7]:
size = 9
train_set = standarize_data_length(train_set,size)
test_set = standarize_data_length(test_set,size)
predict_set = standarize_data_length(predict_set,size)

In [8]:
train_set = standarize_target_vector(train_set,alphabet_lookup)
test_set = standarize_target_vector(test_set,alphabet_lookup)
predict_set = standarize_target_vector(predict_set,alphabet_lookup)

In [9]:
train_target = regex_to_list(train_data)
test_target = regex_to_list(test_data)
categories_lookup = category_encoding(train_target)

In [10]:
categories_lookup

,Output,Label
0,,0
1,-,1
2,0-9,2
3,A-Z,3


In [11]:
train_target = standarize_data_length(train_target,size)
test_target = standarize_data_length(test_target,size)
train_target=standarize_target_vector(train_target,categories_lookup)
test_target=standarize_target_vector(test_target,categories_lookup)

In [12]:
print("Przykładowy input: ")
print(train_set[0])
print("Przykładowy output: ")
print(train_target[0])
print()
print("Liczba danych testowych inputowych: " + str(len(train_set)))
print("Liczba danych testowych outputowych: " + str(len(train_target)))
print("Liczba wejść sieci neuronowej: "+str(len(train_set[0])))

Przykładowy input: 
[6, 9, 1, 3, 11, 11, 0, 0, 0]
Przykładowy output: 
[2, 2, 1, 2, 2, 2, 0, 0, 0]

Liczba danych testowych inputowych: 3000
Liczba danych testowych outputowych: 3000
Liczba wejść sieci neuronowej: 9


In [13]:
train_X = np.array([np.array(xi) for xi in train_set])
train_Y = np.array([np.array(xi) for xi in train_target])

test_X = np.array([np.array(xi) for xi in test_set])
test_Y = np.array([np.array(xi) for xi in test_target])

predict_X = np.array([np.array(xi) for xi in predict_set])

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=size, input_shape=(size,1), return_sequences=True))
model.add(Dense(1))
model.compile(loss="mse",optimizer ="adam",metrics=["mse","accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 9, 9)              396       
_________________________________________________________________
dense (Dense)                (None, 9, 1)              10        
Total params: 406
Trainable params: 406
Non-trainable params: 0
_________________________________________________________________


In [15]:
train_X = train_X.reshape(3000,9,1)
train_Y = train_Y.reshape(3000,9,1)

test_X = test_X.reshape(3000,9,1)
test_Y = test_Y.reshape(3000,9,1)

predict_X = predict_X.reshape(6,9,1)

In [16]:
model.fit(test_X,test_Y,epochs=10)

Epoch 1/10
94/94 [==============================] - 2s 4ms/step - loss: 1.4869 - mse: 1.4869 - accuracy: 0.2512
Epoch 2/10
94/94 [==============================] - 0s 4ms/step - loss: 0.5861 - mse: 0.5861 - accuracy: 0.2892
Epoch 3/10
94/94 [==============================] - 0s 4ms/step - loss: 0.1845 - mse: 0.1845 - accuracy: 0.4316
Epoch 4/10
94/94 [==============================] - 0s 5ms/step - loss: 0.0981 - mse: 0.0981 - accuracy: 0.4818
Epoch 5/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0748 - mse: 0.0748 - accuracy: 0.4811
Epoch 6/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0609 - mse: 0.0609 - accuracy: 0.4812
Epoch 7/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0493 - mse: 0.0493 - accuracy: 0.4815
Epoch 8/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0420 - mse: 0.0420 - accuracy: 0.4811
Epoch 9/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0364 - mse: 0.0364 - accuracy:

In [17]:
predictions=model.predict(predict_X)

In [18]:
rounded_predictions = np.round(predictions)
rounded_predictions = rounded_predictions.astype(np.int)
temp_str = ""
output_list = []
for i in range(len(rounded_predictions)):
  for j in range(len(rounded_predictions[i])):
    for k in range(len(categories_lookup)):
      if rounded_predictions[i,j]==categories_lookup.Label[k]:
        if categories_lookup.Output[k] != "":
          temp_str = temp_str + "[" + categories_lookup.Output[k] + "]"
  output_list.append(temp_str)
  temp_str = "" 

In [21]:
distinct_regexs = set(output_list)
predict_data['prediction'] = output_list
unique_regexs_frequency = predict_data.prediction.value_counts()
unique_regexs_frequency.sort_values(ascending=False)

[A-Z][A-Z][A-Z][0-9][-][-]             2
[A-Z][-][0-9][-][A-Z][0-9]             2
[A-Z][-][-][A-Z][A-Z][-][-][-][0-9]    1
[A-Z][-][-][A-Z][0-9][-][-][-][0-9]    1
Name: prediction, dtype: int64

In [22]:
def regex_accuracy(regexs, data):
  import re
  number_of_regexs = len(regexs)
  all = len(data.text)
  accuracy_table = []
  for i in regexs:
    positives = 0
    pattern=re.compile(i)
    for j in data.text:
      if pattern.match(j):
        positives+=1
    accuracy_table.append((positives/all)/(1/number_of_regexs))
  return accuracy_table     

In [23]:
a = regex_accuracy(distinct_regexs,predict_data)

In [24]:
a

[1.3333333333333333, 1.3333333333333333, 0.0, 1.3333333333333333]